In [1]:
# -*- coding:utf-8 -*-
# 数据预处理
# 数据清洗：过滤掉非居民用电数据，过滤节假日的用电数据
# 缺失值处理：补充缺失的数据
# 三种方法：Lagrange插值法和Newton插值法以及Series自带的interpolate
#1、Lagrange插值法和Newton插值法解决实际问题中关于只提供复杂的离散数据的函数求值问题，通过将所考察的函数简单化，构造关于离散数据实际函数f（x）的近似函数P（x），从而可以计算未知点出的函数值，是插值法的基本思路。
#2、实际上Lagrange插值法和Newton插值法是同一种方法的两种变形，其构造拟合函数的思路是相同的，而实验中两个实际问题用两种算法计算出结果是相同的。
#3、实验所得结果精确度并不高，一方面是因为所给数据较少，另一方面也是主要方面在Win32中C++中数据类型double精度只有7位，计算机在进行浮点运算时截断运算会导致误差。实际问题中，测量数据也可能导致误差。
#4、在解决实际问题中，更多是利用精确且高效的计算机求解。所以解决问题时不仅要构造可求解的算法，更重要是构造合理的可以编写成程序由计算机求解的算法，而算法的优化不仅可以节省时间空间，更能得到更为精确有价值的结果。

In [2]:
import pandas as pd
inputfile = 'missing_data.xls' #输入数据
data = pd.read_excel(inputfile, header=None)
# 判断DataFrame中的空值的位置
sij = []
for j in range(len(data.columns)):
    s1 = data[data.columns[j]].isnull()
    for i in range(len(s1)):
        if s1[i]==True:
            sij.append((j,i))
sij

[(0, 5),
 (0, 12),
 (1, 3),
 (1, 10),
 (1, 19),
 (1, 20),
 (2, 4),
 (2, 7),
 (2, 10),
 (2, 16)]

In [3]:
# <1> 拉格朗日插值法
# 自定义列向量插值函数
# s为列向量， n为被插值的位置， k为取前后的数据个数， 默认为5
from scipy.interpolate import lagrange #导入拉格朗日插值函数
data1 = pd.read_excel(inputfile, header=None,names=['A','B','C'])

def plotinterplate_columns(s, n, k=5):
    y = s[list(range(n-k,n) + list(range(n+1, n+1+k)))]
    y = y[y.notnull()]#剔除空值
    return lagrange(y.index, list(y))(n)#向位置n出插值并返回该插值结果

# 逐个判断每列是否需要插值
lagij = []
for i in data1.columns:
    for j in range(len(data1)):
        if (data1[i].isnull())[j]:
            data1[i][j] = plotinterplate_columns(data1[i],j)
            lagij.append((i,j,data1[i][j]))
            

data1

,A,B,C
0,235.833300,324.034300,478.323100
1,236.270800,325.637900,515.456400
2,238.052100,328.089700,517.090900
3,235.906300,203.462116,514.890000
4,236.760400,268.832400,493.352591
5,237.151181,404.048000,486.091200
6,237.416700,391.265200,516.233000
7,238.656300,380.824100,493.342382
8,237.604200,388.023000,435.350800
9,238.031300,206.434900,487.675000


In [4]:
data1.to_csv('lagrange.csv')
from pandas import Series
Series(lagij).to_csv('lagij.csv')

In [5]:
# <2> 牛顿插值法

import matplotlib.pyplot as plt

"""
@brief:   计算n阶差商 f[x0, x1, x2 ... xn] 
@param:   xi   所有插值节点的横坐标集合                                                        o
@param:   fi   所有插值节点的纵坐标集合                                                      /   \
@return:  返回xi的i阶差商(i为xi长度减1)                                                     o     o
@notice:  a. 必须确保xi与fi长度相等                                                        / \   / \
          b. 由于用到了递归，所以留意不要爆栈了.                                           o   o o   o
          c. 递归减递归(每层递归包含两个递归函数), 每层递归次数呈二次幂增长，总次数是一个满二叉树的所有节点数量(所以极易栈溢出)                                                                                     
"""

def get_order_diff_quot(xi = [], fi = []):
    if len(xi) > 2 and len(fi) > 2:
        return (get_order_diff_quot(xi[:len(xi) - 1], fi[:len(fi) - 1]) - get_order_diff_quot(xi[1:len(xi)], fi[1:len(fi)])) / float(xi[0] - xi[-1])
    return (fi[0] - fi[1]) / float(xi[0] - xi[1])
"""

@brief:  获得Wi(x)函数;
         Wi的含义举例 W1 = (x - x0); W2 = (x - x0)(x - x1); W3 = (x - x0)(x - x1)(x - x2)
@param:  i  i阶(i次多项式)
@param:  xi  所有插值节点的横坐标集合
@return: 返回Wi(x)函数
"""

def get_Wi(i = 0, xi = []):
    def Wi(x):
        result = 1.0
        for each in range(i):
            result *= (x - xi[each])
        return result
    return Wi
"""
@brief: 获得牛顿插值函数
"""

def get_Newton_inter(xi = [], fi = []):
    def Newton_inter(x):
        result = fi[0]
        for i in range(2, len(xi)):
            result += (get_order_diff_quot(xi[:i], fi[:i]) * get_Wi(i-1, xi)(x))
        return result
    return Newton_inter
# 自定义列向量插值函数
# s为列向量， n为被插值的位置，k为取前后的数据个数， 默认为5
def plotnewton_columns(s):
    y = s[s.notnull()]#剔除空值
    Nx= get_Newton_inter(y.index, list(y))         #向位置n出插值并返回该插值结果
    return Nx

import pandas as pd
from scipy.optimize import newton #导入牛顿插值函数
inputfile = 'missing_data.xls' #输入数据
data2 = pd.read_excel(inputfile, header=None,names=['A','B','C'])

newij = []
for i in data2.columns:
    Newton = plotnewton_columns(data2[i])
    for j in range(len(data1)):
        if (data2[i].isnull())[j]:
            data2[i][j] = Newton(j)
            newij.append((i,j,data2[i][j]))


In [6]:
data2.to_csv('newton.csv')
from pandas import Series
Series(newij).to_csv('newij.csv')
newij

[('A', 5, 227.97718197331577),
 ('A', 12, 237.58023935829542),
 ('B', 3, 969.78391168067287),
 ('B', 10, 194.83076304210508),
 ('B', 19, 6553214.130850032),
 ('B', 20, 50150578.655558825),
 ('C', 4, 494.22872791561548),
 ('C', 7, 486.68478798271082),
 ('C', 10, 611.33674198392237),
 ('C', 16, 655.49360854653901)]

In [7]:
# <3> Series自带的iterplote
data3=pd.read_excel('missing_data.xls',header=None,names=['A','B','C'])

###利用Pandas中interpolate进行缺失值的补充
data_out = data3.interpolate()

df = data_out - data3.fillna(0)
Serij = []
for i in df.columns:
    for j in range(len(df)):
        if df[i][j] != 0:
            Serij.append((i,j,df[i][j]))

In [8]:
data_out.to_csv('Series.csv')
Series(Serij).to_csv('Serij.csv')
N_L_S = pd.concat([data1,data2,data_out],axis=1)
N_L_S.to_csv('nls.csv')

In [9]:
## 比较三者的填充结果
N = pd.DataFrame(newij,columns = ['col', 'row', 'value'])
L = pd.DataFrame(lagij,columns = ['col', 'row', 'value'])
S = pd.DataFrame(Serij,columns = ['col', 'row', 'value'])
N.set_index(['col','row'],inplace = True)
L.set_index(['col','row'],inplace = True)
S.set_index(['col','row'],inplace = True)

In [10]:
compare = pd.merge(N,pd.merge(L,S,left_index = True, right_index = True),left_index = True, right_index = True)
compare.rename(columns={'value':'Newton','value_x':'lagrange','value_y':'Series'},inplace= True)
compare.ix[:,:] = compare.ix[:,:].applymap(lambda x: '%.3f' % x)  # 一旦执行这个操作，数据类型转成字符串型
# compare['Newton'] = compare['Newton'].astype('float64') # 这一句不能要，否则又恢复原样了
# compare = compare.round(3)# 此句话对科学计数法没起作用
compare.to_csv('compare.csv')
compare

D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


Newton lagrange   Series
col row                                
A   5         227.977  237.151  237.089
    12        237.580  235.315  235.000
B   3         969.784  203.462  298.461
    10        194.831  237.348  303.257
    19    6553214.131  420.749  416.880
    20   50150578.656  408.963  416.880
C   4         494.229  493.353  500.491
    7         486.685  493.342  475.792
    10        611.337  609.194  573.955
    16        655.494  618.197  622.641